In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import datetime
import random

#from api_keys import api_key

#url = f"https://api.nal.usda.gov/ndb/reports/?"


In [2]:
#Creación del formulario para el usuario

user_dict = {
    "First_Name":'',
    "Last_Name": '',
    "Age":'',
    "Sex":'',
    "Weight_in_Kilograms":'',
    "Height_in_meters":''
    }



for key in user_dict:
    user_dict[key] = str(input(f"Please type your {key} "))

activitie = str(input(f"Please type your type of activitie you have: Sedentary (S), Low Active (L), Active (A) or Very Active (V) "))

if user_dict["Sex"] == "Women":
    pregnant_q = str(input(f"Please type if you are pregnant Yes(Y)/ No(N) (Only the letter) "))
else:
    pregnant_q = "N/A"

if pregnant_q == "Y":
    pregnant_a = int(input(f"In wich trimester of pregnancy are you (Only the number) ? "))
else:
    pregnant_a = "N/A"
    
if user_dict["Sex"] == "Women":
    lactation_q = str(input(f"Please type if you are in lactation Yes(Y)/ No(N) (Only the letter)"))
else:
    lactation_q = "N/A"

if lactation_q == "Y":
    lactation_a = int(input(f"Please type if you are inside the 1st or 2nd Semester of lactation (Only put the number) "))
else:
    lactation_a = "N/A"

Please type your First_Name Alejandro
Please type your Last_Name Dorantes
Please type your Age 26
Please type your Sex Men
Please type your Weight_in_Kilograms 77
Please type your Height_in_meters 1.77
Please type your type of activitie you have: Sedentary (S), Low Active (L), Active (A) or Very Active (V) S


In [3]:
#Cálculo del IMC actual y el ideal.

IMC_actual = round(float(user_dict["Weight_in_Kilograms"])/(float(user_dict["Height_in_meters"])**2),2)

if (IMC_actual < 24.9 and IMC_actual > 18.5):
    IMC_ideal = IMC_actual    
if IMC_actual > 24.9:
    IMC_ideal = 24.5
if IMC_actual < 18.5:
    IMC_ideal = 19
    
Ideal_Weight = round(IMC_ideal * (float(user_dict["Height_in_meters"])**2),1)

Variance_Weight = round(float(user_dict["Weight_in_Kilograms"]) - float(Ideal_Weight),1)

print(IMC_actual)
print(Ideal_Weight)
print(Variance_Weight)

24.58
77.0
0.0


In [4]:
#Cálculo de la actividad física, para la fórmula de EER

pa_data_df = pd.DataFrame({"Category": ["S", "L", "A", "V"],
                          "Men": [1,1.11,1.25,1.48],
                          "Woman": [1,1.12,1.27,1.45]})

pa_data_df = pa_data_df.set_index("Category")

if user_dict["Sex"] == "Men":
    P_Activitie = pa_data_df.loc[activitie,"Men"]
else:
    P_Activitie = pa_data_df.loc[activitie,"Woman"]


#Cálculo del requerimiento de energía diario de una persona.

EER_Men= float(662) - (9.53 * float(user_dict["Age"]))+ float(P_Activitie) *((15.91*float(Ideal_Weight)) + (539.6*float(user_dict["Height_in_meters"]))) 
EER_Women= float(354) - (6.91 * float(user_dict["Age"]))+ float(P_Activitie) *((9.36*float(Ideal_Weight)) + (float(726)*float(user_dict["Height_in_meters"])))

    
if user_dict["Sex"] == "Men":
    Nut_Req_Atnow = round(EER_Men,2)
else:
    Nut_Req_Atnow = round(EER_Women,2)
    if pregnant_q == "Y":
        if pregnant_a == 1:
            Nut_Req_Atnow = Nut_Req_Atnow
        if pregnant_a == 2:
            Nut_Req_Atnow = Nut_Req_Atnow + 340
        if pregnant_a == 3:
            Nut_Req_Atnow = Nut_Req_Atnow + 452
    if lactation_q == "Y":
        if lactation_a == 1:
            Nut_Req_Atnow = Nut_Req_Atnow + 600 - 170
        if lactation_a == 2:
            Nut_Req_Atnow = Nut_Req_Atnow + 400
            

print(f"{Nut_Req_Atnow} Kcal")

2594.38 Kcal


In [5]:
#Distribución de la energía en carbohidratos, grasas y proteina

Carb_Dist_kcal= round(Nut_Req_Atnow * .50,0)
Fat_Dist_kcal= round(Nut_Req_Atnow * .3,0)
Protein_Dist_kcal = round(Nut_Req_Atnow * .2,0)

print(Carb_Dist_kcal)
print(Fat_Dist_kcal)
print(Protein_Dist_kcal)

1297.0
778.0
519.0


In [6]:
#Conversión de Kcal a gramos

Carb_Dist_gr= round(Carb_Dist_kcal / float(4),0)
Fat_Dist_gr= round(Fat_Dist_kcal / float(9),0)
Protein_Dist_gr = round(Protein_Dist_kcal / float(4),0)

print(Carb_Dist_gr)
print(Fat_Dist_gr)
print(Protein_Dist_gr)

324.0
86.0
130.0


In [7]:
User_Information=pd.DataFrame({"Name": user_dict["First_Name"],
                              "Last Name": user_dict["Last_Name"],
                               "Age": user_dict["Age"],
                               "Sex": user_dict["Sex"],
                               "Weight (Kg)": user_dict["Weight_in_Kilograms"],
                               "Height (m)": user_dict["Height_in_meters"],
                               "Pregnant": pregnant_q,
                               "Pregnancy Trimester": pregnant_a,
                               "Lactation": lactation_q,
                               "Lactation Semester": lactation_a,
                               "Actual IMC": IMC_actual,
                               "Ideal IMC": IMC_ideal,
                               "Ideal Weight(Kg)": Ideal_Weight,
                               "Extra/Missing Weight (Kg)": Variance_Weight,
                               "Physical Activitie": activitie,
                               "Energy Requirement (kcal)": Nut_Req_Atnow,
                               "Carbohydrates Req (kcal)": Carb_Dist_kcal,
                               "Fat Req (kcal)": Fat_Dist_kcal,
                               "Protein Req (kcal)": Protein_Dist_kcal,
                               "Carbohydrates Req (gr)": Carb_Dist_gr,
                               "Fat Req (gr)": Fat_Dist_gr,
                               "Protein Req (gr)": Protein_Dist_gr
                              }, index=[0])

pd.options.display.max_columns=22


User_Information

,Name,Last Name,Age,Sex,Weight (Kg),Height (m),Pregnant,Pregnancy Trimester,Lactation,Lactation Semester,Actual IMC,Ideal IMC,Ideal Weight(Kg),Extra/Missing Weight (Kg),Physical Activitie,Energy Requirement (kcal),Carbohydrates Req (kcal),Fat Req (kcal),Protein Req (kcal),Carbohydrates Req (gr),Fat Req (gr),Protein Req (gr)
0,Alejandro,Dorantes,26,Men,77,1.77,N/A,N/A,N/A,N/A,24.58,24.58,77.0,0.0,S,2594.38,1297.0,778.0,519.0,324.0,86.0,130.0


In [8]:
# Leer el CSV

nutrients_data = 'Resources/Nutrients.csv'
nutrientsdf = pd.read_csv(nutrients_data, encoding="ISO-8859-1")
nutrientsdf.head()

,NDB_No,Nutrient_Code,Nutrient_name,Derivation_Code,Output_value,Output_uom
0,45127487,203,Protein,LCGA,4.3,g
1,45127487,204,Total lipid (fat),LCGA,18.9,g
2,45127487,205,"Carbohydrate, by difference",LCGA,72.8,g
3,45127487,207,Ash,LCGA,2.0,g
4,45127487,255,Water,LCGA,2.0,g


In [9]:
# Base de Nutrientes solo con Proteínas, Carbohidratos y Grasas

nutrients_loc=nutrientsdf.loc[nutrientsdf["Nutrient_Code"].isin([203,204,205])] 
nutrients_loc.head()


,NDB_No,Nutrient_Code,Nutrient_name,Derivation_Code,Output_value,Output_uom
0,45127487,203,Protein,LCGA,4.3,g
1,45127487,204,Total lipid (fat),LCGA,18.9,g
2,45127487,205,"Carbohydrate, by difference",LCGA,72.8,g
29,45128706,203,Protein,LCGA,10.1,g
30,45128706,204,Total lipid (fat),LCGA,8.5,g


In [10]:
# Leer el CSV de Productos

products_data = 'Resources/Products.csv'
productsdf = pd.read_csv(products_data, encoding="ISO-8859-1")
productsdf.head()

C:\Users\AlejandroDorantes\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,NDB_Number,long_name,data_source,gtin_upc,manufacturer,date_modified,date_available,ingredients_english
0,45001524,MOCHI ICE CREAM BONBONS,LI,19022128593,"G. T. Japan, Inc.",2017-11-15 19:19:38,2017-11-15 19:19:38,"ICE CREAM INGREDIENTS: MILK, CREAM, SUGAR, STR..."
1,45001528,CHIPOTLE BARBECUE SAUCE,LI,5051379043735,FRESH & EASY,2018-04-26 17:23:31,2018-04-26 17:23:31,"WATER, SUGAR, TOMATO PASTE, MOLASSES, DISTILLE..."
2,45001529,HOT & SPICY BARBECUE SAUCE,LI,5051379009434,FRESH & EASY,2018-04-26 18:17:37,2018-04-26 18:17:37,"SUGAR, WATER, DISTILLED VINEGAR, TOMATO PASTE,..."
3,45001530,BARBECUE SAUCE,LI,5051379019969,FRESH & EASY,2018-04-26 17:24:00,2018-04-26 17:24:00,"TOMATO PUREE (WATER, TOMATO PASTE), SUGAR, DIS..."
4,45001531,BARBECUE SAUCE,LI,5051379009526,FRESH & EASY,2018-04-26 17:47:41,2018-04-26 17:47:41,"SUGAR, DISTILLED VINEGAR, WATER, TOMATO PASTE,..."


In [11]:
# Obtener 25 alimentos aleatorios que se consumen en un día
Fats_Month=[]
Carbs_Month=[]
Proteins_Month=[]
Days=range(1,31)

for days in Days:
    
    selected_products = productsdf.sample(n=25)
    products_nutrients = selected_products.loc[:,["NDB_Number", "long_name"]]
    products_index = products_nutrients["NDB_Number"].unique()
    nutrients_sample=nutrients_loc.loc[nutrients_loc["NDB_No"].isin(products_index)]
    nutrients_proteins = nutrients_sample.loc[nutrients_sample["Nutrient_Code"] == 203]
    proteins = nutrients_proteins.loc[:,["NDB_No", "Output_value"]]
    proteins = proteins.rename(columns = {"NDB_No": "NDB_Number"})
    nutrients_fats = nutrients_sample.loc[nutrients_sample["Nutrient_Code"] == 204]
    fats = nutrients_fats.loc[:,["NDB_No", "Output_value"]]
    fats = fats.rename(columns = {"NDB_No": "NDB_Number"})
    nutrients_carbs = nutrients_sample.loc[nutrients_sample["Nutrient_Code"] == 205]
    carbs = nutrients_carbs.loc[:,["NDB_No", "Output_value"]]
    carbs = carbs.rename(columns = {"NDB_No": "NDB_Number"})
    products_nutrients = pd.merge(products_nutrients, proteins, on = "NDB_Number")
    products_nutrients = products_nutrients.rename(columns = {"Output_value": "Proteins (gr)"})
    products_nutrients = pd.merge(products_nutrients, fats, on = "NDB_Number")
    products_nutrients = products_nutrients.rename(columns = {"Output_value": "Total Lipid (fat) (gr)"})
    products_nutrients = pd.merge(products_nutrients, carbs, on = "NDB_Number")
    products_nutrients = products_nutrients.rename(columns = {"Output_value": "Carbohydrate (gr)"})
    total_protein = sum(products_nutrients["Proteins (gr)"])
    total_fats = sum(products_nutrients["Total Lipid (fat) (gr)"])
    total_carbs = sum(products_nutrients["Carbohydrate (gr)"])
    Fats_Month.append(total_fats)
    Carbs_Month.append(total_carbs)
    Proteins_Month.append(total_protein)
    
Monthly_Result=pd.DataFrame({"Days": Days,
                            "Carbohydrates Consumed": Carbs_Month,
                            "Fats Consumed": Fats_Month,
                            "Proteins Consumed": Proteins_Month})

Monthly_Result

    
    



,Days,Carbohydrates Consumed,Fats Consumed,Proteins Consumed
0,1,939.37,195.13,179.79
1,2,611.11,496.32,155.18
2,3,1125.38,288.48,170.23
3,4,651.53,307.75,183.49
4,5,1006.29,289.00,199.15
5,6,877.63,372.01,238.82
6,7,757.55,354.24,191.37
7,8,833.81,325.25,185.58
8,9,731.58,358.76,140.83
9,10,876.19,198.12,136.87
